<center> 
    <h1> 
        Projet Compressed Sensing - Mars 2021
    <h2> 
        ENSAE 3e année - Cycle Ingénieur

<center>
<img src="https://drive.google.com/uc?id=11e9TZk9co2fqaHFq8RJkQ3ggzyPgVNBW" width="200px">

 <center>
    <h2> 
        Article : <br>
        Recurrent Neural Network with Top-k Gains for Session-based Recommendations

<div style="text-align: right">
    <em>
        Auteurs (article) : Balazs Hidasi - Alexandros Karatzoglou <br>
        Auteurs (rapport) : Bastien Billiot - Joël Garde <br>
        Professeur : Guillaume Lecué <br>
        Link to the RSC15 dataset that will be used in this notebook : https://2015.recsyschallenge.com/challenge.html
    </em>
</div>

# Part 1 : Introduction

## Context of the study and link to the Compressed Sensing course

*  **Introduction to Recommandation Systems** 

The emergence of new business models based on products and services digitalization to reduce time and distance pain points of the customer has led to an increase of online interactions with the customers. Sectors such as the hotel industry, retail, video and music services have therefore been disrupted by such business models. For companies proposing those online offers, retaining customers has become crucial and is played out through traditional assets such as quality, depth of supply or online assets such as delivery quality and rapidity. However to differentiate from other online competitors, additional services and features have become crucial to increase customer retention. One of the most important of these features is a qualitative and relevant recommandation systems which can be defined in simple terms as systems built to predict what users might like, especially when there are lots of choices available. 

These systems of recommandation can be separated in two different categories. Collaborative filtering recommandation systems make recommandations for a target user based on other users' actions or choices i.e. groups of users can be created based on their activities or likings and recommandations for a recent user for which little information is available is made based on popular choices within the target user group. Another group of recommandation systems is Content-Based. These systems are based on the target user's past history of choices, actions and inputs.

<center><img src="https://drive.google.com/uc?id=1gCTG0DidqPGrmJN57WzF2w-oLpmKRViD" width="600px" align="center"></center>  

*  **Link with Compressed Sensing : Recommandation System and matrix completion**

In class, we have thus seen the case of collaborative-filtering recommandation system with the Netflix Prize where the data available and to recommend are summarized in a user/item matrix such as the following one.   

<center><img src="https://drive.google.com/uc?id=1cCc209_TlI4yTwWnFvWgm9GeR-iRHoQu" width="400px"></center>


In the Netflix Prize recommendation problem, this user/item matrix is however larger with about 500 000 users (rows) and 18 000 items (movies). In the course, we have seen an aproach to solve this collaborative-filtering problem through the use of a matrix completion procedure allowing to retrieve the missing/unseen movie ratings a user would give and thus to recommend them or not to the user. 
The idea in the context of the Netflix Prize is that users as well as movies can be categorized into homogeneous groups (collaborative-filtering) thus leading to the idea of a low-rank matrix. Thus, the problem can be handled through a rank-minimization procedure. The problem can then be formulated as follows : 

<h6><center>Rebuild $A^* \in \mathbb{R^{u \times v}}$ with the availbale data of the matrix $Y_i = (<A^*,X_i>)_{i=1,...,m}$, knowing that $A^*$ is low-rank</center></h6>

and the solution is to solve the rank-minimization procedure : 

<h6><center> $\min \limits_{A} (rg(A) : <A, X_i> = <A^*, X_i>, i=1,...m) $</center></h6>

This problem being NP-hard, we can get by convex relaxation (convex hull of $rg$ function) that this procedure to solve matrix completion can be reduced to the nuclear norm minimization problem as follows : 

<h6><center> $\min \limits_{A} ( ||A||_{S_1} : <A,X_i>=Y_i, i=1...,m) $</center></h6>

We can then solve the NNM using Semi Definite Programming. Therefore we can solve this collaborative-filtering problem (Netflix Prize) through a matrix completion algorithm such as the two seen in class : the projected gradient descent and the proximal gradient descent. 

## Presentation of the problem

*  **Introduction to session based recommandation systems**

The procedure seen in class offers an efficient solution to the problem of recommandation system using matrix completion. However, in many cases, users cannot be identified (users not logged in or first-time users...). 

<b> <i>Session-based recommandation system </b></i> has therefore emerged in recent years as a new aspect of recommender systems, in which, as opposed to a user-based recommendation system, recommendations rely only on the actions of the current session and have no past history or metadata specific to the current user. Such recommandation systems have grown to be important for many business sectors such as music, video, online retail where the current user is not logged through any subscribed account for instance. 

Additionnaly, as in many other fields, Deep Learning models have taken over precedently used methods in recommendation systems thanks to their higher efficiency. In the context of session-based recommandation systems, Recurrent Neural Networks appear as a "natural choice" for the problem as they learn from sequential data which matches a session of events online. 

*  **Introduction to the paper studied**

In this paper, the two authors Hidasi and Karatzoglou study methods that have been developped in session-based recommandation systems. More precisely, they continue their previous work [*Session-Based Recommendations with Recurrent Neural Networks*](https://arxiv.org/pdf/1511.06939.pdf) (2016) in which they  proposed an implementation of a Recurrent Neural Network (RNN) called **GRU4REC**. 

They come back on this previous model and propose two areas of improvement, namely the **sampling method** and the **loss functions**, building up from their previous publication.

In this first part of our report we propose to introduce more deeply the problem of session-based recommandation systems, the general handling and method presented by the authors, as well as an introductory explanation of their GRU4REC model proposed in their previous paper. 



```
┌───────┬───────┬──────┐
│ ITEM_1│ ITEM_2│ITEM_3│
└───────┴───────┴──┬───┘   
    SESSION        └────────► RECOMMANDATION
```

## Problem definition and model introduction

We study Session Based Recommandations. A session $s \in S$ is a sequence of events $a_i \in V$.

$$
s = [ a_1, a_2, \dots, a_n ]
$$

Notice that the events are ordered, and that two different sessions might have different lenghts.

Given an ongoing session $s = [ a_1, a_2, \dots, a_n ]$, the goal is to provide the user with a relevant recommandation for what the next event $a_{n+1}$ could be. For instance, considering events to be clicks on products in a e-commerce setting, the session based recommandation wishes to present to the user products he is highly likely to engage with and buy.

At time step $t$, given and depending on $[a_0, \dots, a_{t-1}]$, we wish to provide (output of the trained model) a **ranking** over all the possible events in $V$, so that events are ordered by their probability of being performed by the user. More precisely, we are only interested in the ranking of the $k$ first recommanded items. For example, events could be visiting websites, $k$ could be the number of links in the first page of a web search, and the ranking determines the order of the websites in the search page.Therefore, we wish that the model outputs, for a session $[a_0, \dots, a_{t-1}]$, the ranking of all items in the catalog $V$. Then, to evaluate the model, we only take into account the $k$ highest ranked items and two aspects are important to evaluate our model : that the groundtruth item (the item the user actually selected) is among the $k$ predicted items and that it is ranked the highest.

There are three main challenges to overcome:

1. Sessions are ordered sequences of different lengths, meaning the model needs to be flexible.
2. The events set $V$ has a high cardinality. $|V| \approx 10^4\;\text{to} \;10^6$ (for datasets usually used in session-based recommandation systems, see Figure below extracted from this [evaluation paper](https://arxiv.org/pdf/1803.09587.pdf))
3. The evaluation metrics are ranking ones (Recall@20, MRR@20, see Results), which are unusual in machine learning.

<center><img src="https://drive.google.com/uc?id=1nHIoTAYzA3vMJH7JgXleX_JujJLCRcCk" width="600px"></center>

## Motivation 

The authors set Recurent Neural Networks (RNNs) as the best models to simulate a whole session of user interactions, using as justification their higher performances (referencing their results from their previous article against standard methods such as k Nearest Neighbours). Indeed, RNNs can capture time dependant information and thus are adapted to the sequencial nature of users' sessions. Therefore, the authors decide to consider their previous work, with an already well-performing model on some datasets, as the context of this paper. They therefore propose areas of improvements. However, before developing those areas, we propose a brief introduction to the GRU4REC model which serves as a basis of our article and thus of our discussion. Those elements are taken from the previously cited work of the authors [*Session-Based Recommendations with Recurrent Neural Networks*](https://arxiv.org/pdf/1511.06939.pdf) (2016)

>*Remark : when the original GRU4REC paper came out (2016), GRUs (1014) were state-of-art for sequential learning. Today, some architectures based on transformers are picking up.*

## The GRU4REC model

### The architecture 

GRU4REC implements a form of **sequence modelling** where the model learns to predict the next event in a session. Given a sequence $s = [a_1, \dots, a_n]$, it outputs a probability distribution over $V$ for the next event $a_{n+1}$.

GRU4REC is an RNN (Recurrent Neural Network) based deep learning model. Its uses a GRU (Gated Recurrent Unit) as its heart. 

In its classical version GRU4REC is composed of 1 GRU cell with an hidden dimention of 100.



                  -----------------+  INPUT a_i
                  |
                  |
        +---------v---------+
        | ITEM TO EMBEDDING |
        | MATRIX            |
        +-------------------+
                  |
        +---------v---------+
        | GRU UNIT          +<--+
        +-------------------+   |
                  |             |   RECURRENT h_i
                  +-------------+
                  |
                  |
        +---------v---------+
        | EMBEDDING TO ITEM |
        | MATRIX            |
        +-------------------+
                  |
                  |
                  +---------------->  OUTPUT probabilities for a_i+1


At each step, the model is fed the current event, $a_i$, and a vector representing the internal state of the GRU cell $h_i$. It outputs the next state of the cell, $h_{i+1}$, which will be used to predict the next event $a_{i+1}$, and also be fed at the next time step.

In [ ]:
# an implementation of the GRU4REC architecture in pytorch.
class GRU4REC(torch.nn.Module):
  """GRU4REC architecure with tied weights"""
  
  def __init__(self, n_h, n_voc):
    """
    n_h: number of hidden units.
    n_voc: size of the vocabulary.
    """
    super().__init__()
    self.n_h = n_h
    self.n_voc = n_voc
    self.embedding = torch.nn.Parameter(torch.empty((n_voc, n_h)))
    torch.nn.init.xavier_uniform_(self.embedding, gain=torch.nn.init.calculate_gain('relu'))
    self.gru = torch.nn.GRUCell(n_h, n_h)
  
  def forward(self, inputs, carry):
    h = self.embedding[inputs] # embedding step
    h = self.gru(h, carry) # GRU step.
    return h
  
  def embed(self, inputs):
    return self.embedding[inputs]

###  The training procedure

GRU4REC is trained on mini-batches of sequences slices: $b_s$ sequences (sessions) are fed in parallel to the model at a time, event per event. The model is trained to predict the next event in each session.

This process of mini-batching is mainly done to fully use the parallel processing capabilities of GPUs.

This type of mini-batch is called $bptt(1, 1)$ (backpropagation-through-time) in the RNN litterature: the model passes over 1 step of the $b_s$ sequences, the gradient step is performed, and the sequences are then moved 1 step further.

<center>
<img src="https://drive.google.com/uc?id=1sTPAA3Vzic7bg2ngNn1O4gIlfoSUyoUX" width="600px">

In [ ]:
# a dataset of mini-batches of sessions as defined above.
class SessionBatchDataset(torch.utils.data.IterableDataset):
  """ a dataset of minibatch as defined in GRU4REC.
  """
  def __init__(self, data:utils.SessionData):
    super(self, SessionBatchDataset).__init__()
    for k,v in asdict(data).items():
      setattr(self, k, v)
    
  def __iter__(self):
    r = np.random.permutation(len(self.off) - 1)
    n_new = ScriptParams().bs
    n_done = 0
    cs = np.zeros(n_new, dtype=np.int) # current session index in the dataset.
    so = np.zeros_like(cs) # session offsets
    reset = np.ones_like(cs, dtype=bool)

    while n_new + n_done < r.shape[0]:
      cs[reset] = r[n_done:n_done+n_new] # start new session
      n_done += n_new
      CUR = self.offsets[cs] + so # current offset in the current session
      END = self.offsets[cs+1] - 1
      TARG = CUR + 1 # the target is the next item
      b = self.data[torch.stack([CUR, TARG])] # we batch the feature and target together
      reset = (TARG == END) # if we finish the session, we need to reset the state.
      so += 1 # we have done 1 event, we advance by one in the session.
      so[reset] = 0 # if the session are done, we will start new one at the beginning.
      n_new = reset.sum() #how many session we have done at this step.
      yield b, reset

# Part 2 : Discussion on the paper 

In this second part of the paper, we propose to summarize the key motivations, procedures and ideas of the authors which are developed in this specific article. Moreover, we propose some first personal inputs with a development of problematic points and further development. 

## Motivations : Loss and sampling improvements

The first motivation that is pointed out is the need to refine the previously defined loss function used in the GRU4Rec model. Indeed, the backpropagation through time used in RNNs and GRUs to learn the model parameters leads to vanishing gradients, inhibiting efficient learning. The choice of a loss function with derivatives which are resistant to vanishing gradients is therefore of importance. A study of the previously defined functions and their gradients is thus the motivation to look for improvements. 

Another motivation for this publication  is to propose a method to sample as effectively as possible the large output space present in the recommandation system task as a result of the large number of proposed items. Indeed, using the full output space when computing the loss is prohibitly expensive.  

## Sampling procedure and improvements


### Why do we need sampling

At each step of the learning procedure, GRU4REC takes a batch of events and outputs a set of scores, each score represents how likely an event is to be the next in the sequence.

Given the high cardinality of the events sets $V,  (|V| \approx 10^4\;\text{to} \;10^6)$, it is highly impractical to compute the scores of all the items in $V$.
If the network takes as input the current event of the session (the item the session's user is on) as a one-hot encoded vector and outputs the likelihood for each and every item to be the next one in the session, as the network will iterate at each step during training through all examples, computing scores for all items will become computationally costly.

Instead, we **subsample** a subset of the output, and compute scores only on this subset.

> A famous machine learning algorithm that also does next-event prediction and subsamples the output is [word2vec](https://arxiv.org/pdf/1301.3781.pdf). Word2vec and GRU4REC are very similar. Their main difference is the GRU cell in the GRU4REC architecture. A lot of litterature on how to sample the output came from word2vec.



                                                                                +-----------+-----------+-----------+
                                                                                |           |           |           |
                                                                                |   SCORE 1 |   SCORE 1 |   SCORE 1 |
                                                                                |           |           |           |
                    +-------+                                                   +-----------------------------------+
                    |       |                                                   |           |           |           |
          SESSION A |ITEM i |                                                   |   SCORE 2 |   SCORE 2 |   SCORE 2 |
                    |       |      +---------------------------+                |           |           |           |
                    +-------+      |                           |                +-----------------------------------+
                    |       |      |                           |                |           |           |           |
          SESSSION..| ....  +------>     GRU4Rec               +--------------->+   SCORE 3 |   SCORE 3 |   SCORE 3 |
                    |       |      |                           |                |           |           |           |
                    +-------+      +---------------------------+                +-----------------------------------+
                    |       |                                                   |           |           |           |
                    |       |                                                   |   SCORE...|   SCORE...|   SCORE...|
          SESSION N |ITEM k |                                                   |           |           |           |
                    |       |                                                   +-----------------------------------+
                    +-------+                                                   |           |           |           |
                                                                                |   SCORE...|   SCORE...|   SCORE...|
                                                                                |           |           |           |
                                                                                +-----------------------------------+
                                                                                |           |           |           |
                                                                                |   SCORE V |   SCORE V |   SCORE V |
                                                                                |           |           |           |
                                                                                +-----------+-----------+-----------+

                                                                                 SESSION A   SESSION...  SESION N


### Negative sampling

The losses used to train the network are listwise ranking loss functions that compare the score of the true next item (target) to items that are not the next one (called negative sample). Therefore, at each training steps, we compute the score for the target item and for a sample of negative items.

For the training to be efficient, the difference between the target's score and the negative sample's score must not be too high: the negative samples need to include high scoring ones.
We will call these high-scoring negative samples relevant.
Karatzoglou and Hidasi explain this fact in terms of vanishing gradients. Low scoring negative samples do not contribute to the gradient. And because the gradient is averaged over negative samples, they push the whole gradient to zero. 

> There is an intuitive explanation of this requirement. If negative samples scores are low, the model can easily discriminate between them and the target. If the task is easy for the model, it has nothing to learn! If the model only sees easy examples, it will never learn to discriminate the hard ones. 



### Original sampling procedure

The authors firstly present the method used in their previous paper: mini-batch sampling. In mini-batch sampling, we choose a number $N_B$ of items to select. $N_B$ is called the mini-batch size. Among the samples, we have the current item and the $N_B-1$ remaining elements are the negative samples. 

Mini-batch sampling has two advantages:

* It is really efficient to implement : the negative samples that are used for mini-batch sampling are taken from the input mini-batch (see training procedure) i.e. there is no need to sample additional negative samples.
* Consider a high-frequency event in the dataset. Because it appears often, it is also often the next item to predict, and therefore will statistically have a high score. Now, because this very event is frequent, it will also often be in the mini-batch, and because it is in the mini-batch, it will be a negative sample for all the other events in the same mini-batch. Therefore, frequent events will also be high-scoring negative samples. They are relevant negative samples.



```
                     +---------------------+
                     |TARGET SCORE         |
                     | i  j  k  ... l      |
+------------------------------------------+
|SESSION 1  ||ITEM i|| 1  0  0  ... 0      |
|           ||      ||                     |
|SESSION 2  ||ITEM j|| 0  1  0  ... 0      |
|           ||      ||                     |
|SESSION... ||ITEM k||       \\\           |
|           ||      ||                     |
|SESSION 32 ||ITEM l|| 0  0  0 ...   1     |
+------------------------------------------+
```



<center> <i> For each session, the positive sample is the one corresponding to the same number (the diagonal). All others are considered negative samples (the zeros). </center> </i> 

### Proposed Sampling procedure


Hidasi and Karatzoglou propose an improved method over mini-batch-sampling as it also presents three main drawbacks:
* The mini-batch does not always contain high scoring negative samples as the output space (number of items) is often very large and the mini-batch size relatively small. ($B_s \approx 32, |V| \approx 10^5$)
* The smaller the mini-batch size is, the more acccurate is the model which reduces the benefits of using mini-batch compared to stochastic method. 
* According to the authors the mini-batch sampling might not be optimal for all datasets.

To cope with those drawbacks, they propose to sample $N_A (\approx 2048)$ additional negative samples in addition to mini-batch-sampling. Those additional examples are drawn only once and will be shared by all mini-batches. $N_B(\approx 32)$ input elements are sampled in each mini-batch and the score is then computed between the target example and the $(N_B - 1) + N_A$ elements.

Overall, the target is thus compared to $2079$ negative samples, which is way more than the previous $31$ using only mini-batch-sampling, yet an order of magnitude less than the $\approx 30 000$ total possibilities.

**Sampling Distribution**

With the above procedure, $N_A$ negative samples must be sampled from a distribution $Q$. Two question arise, which did not with mini-batch sampling only:

1. How to choose $Q$?
2. How to implement efficiently the sampling?

The sampling distribution $Q$ must be chosen so that high scoring negative samples are more likely to be sampled. The authors choose 

$$Q(i) = supp(i)^\alpha$$

Where $\alpha$ is a hyper-parameter, and $supp(i)$ is the support of the ith item, determined from the frequencies in the training dataset.
This way, frequent items are more likely to be sampled than others. The authors tune $\alpha$ for best performance.

<center>
<img src="https://drive.google.com/uc?id=1TeY7nJRUpyGYGfJKgS4U3eGSIjoaP6iz" width="800px">

<center> <i> Graph of the frequencies of an item in the training dataset RSC15. </center> </i> 

<center> <i> For $\alpha = 1$, we get the unigram distribution where the probabilities are the normalized frequencies. For $\alpha = 0$, we get a uniform distribution. In the above graph, by choosing $\alpha = 0.5$, the distribution is smoothed: higher frequencies are damped. 

Hidasi and Karatzoglou are also interested in how to sample efficiently from $Q$, meaning with a computational time as small as possible. There is a tradeoff between sampling on CPU and transfering to GPU, or sampling directly on GPU, which does not need to transfer the data from system memory to the GPU memory but where there is no really fast implementation of the sampling.

The authors choose to precompute a buffer of samples, which is transfered to GPU at the beginning of the training. During the training, samples are simply picked in the buffer.

>The distribution $Q$ is called the $\alpha$ power-raised unigram distribution in word2vec.

> Fast sampling on GPU is a complex issue.

In [ ]:
# fast negative sampling in pytorch.
class RandomDataset(torch.utils.data.IterableDataset):
  """ a dataset of random negative samples.
  """
  def __init__(self, n_neg, p):
    self.n_neg = n_neg
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.p = torch.from_numpy(p).to(self.device)
    self.cs = torch.cumsum(self.p, dim=0) # get the cumulative probability distribution.
  def __iter__(self):
      while True:
        yield torch.searchsorted(self.cs, torch.rand(self.n_neg,device=self.device)) # trick for faster sampling.



```
                     +-----------------------------------------+
                     |TARGET SCORE         |NEGATIVE SAMPLES   |
                     | i  j  k  ... l      | 1  2  3  4 ... NA |
+--------------------------------------------------------------+
|SESSION 1  ||ITEM i|| 1  0  0  ... 0      | 0  0  0  0 ... 0  |
|           ||      ||                     |                   |
|SESSION 2  ||ITEM j|| 0  1  0  ... 0      | 0  0  0  0 ... 0  |
|           ||      ||                     |                   |
|SESSION... ||ITEM k||       \\\           |       \\\         |
|           ||      ||                     |                   |
|SESSION 32 ||ITEM l|| 0  0  0 ...   1     | 0  0  0  0 ... 0  |
+------------------------------------------+-------------------+
```



> Between 2015 (first paper) and 2018 (second paper) a lot of improvements were made in GPUs. In 2015, mini-batch sampling (adding 2048 additional samples for the training as explained before) was surely justified , but the hardware memory available on GPUs was probably not sufficient enough. However, the advances in hardware made it possible to increase the number of negative samples.

## Loss improvement procedure


### Previous loss procedure

If we consider the problem at hand as a problem of next item recommandation then the problem considered can be seen as a multi-classes classifiation. Indeed each item $i$ of the catalog $V$ which can be proposed can be seen as a class. If we have only one item to predict, the output is then a one-hot encoded vector  of size $|V|$ where the predicted class coordinate (next item recommended) is 1 and the rest are zeros. 

*   **Pairwise and listwise loss functions**

The problem at hand is a ranking one. For that matter, we can use pairwise loss functions as we we want to compare the score of the target item to a negative example. Comparing those scores will enable to extract a ranking of the items compared. In this setting, we remind that it is important with these losses that the score of the target item does not exceed the score of the negative exmaple by a too large margin, otherwise there is nothing to be learnt. For each target item, a pairwise function will be computed between the considered target and each sampled negative examples. Then the value of the loss function for each target item will be an average of those pairwise functions. Therefore, this will result in a listwise loss function : for each coordinate (item in $V$ which is defined as the target) we compute the average of pointwise functions comparing the score of this target to each negative example. 

In their previous work, as well as reminded in our article, Hidasi and Karatzoglou propose three different loss functions that we will present here as well as their drawbacks. 

*   **Cross-entropy with softmax scores**

A commonly used loss function for the problem of multi-class classification in traditional machine or deep learning approaches is the cross entropy. Indeed, it enables to evaluate the distance between two distributions. If we consider a discrete target distribution $\hat{y}$ and a discrete output (predicted) distribution $y$ which is to be compared we then have : 
$$
CE(\hat{y},y) = - \sum \limits_{k=1}^{N} \hat{y}_k log(y_k)
$$
So following one-hot encoding, if we consider $i$ as the index of the target item (the actual next item in the training session), we have $\hat{y}_i = 1$ and $\forall k \neq i$ $\hat{y}_i = 0$. Thus we have the cross-entropy with sotftmax : 
$$
L_{ce-softmax} = -log(s_i)
$$
It is a listwise loss function with a separation for each target item. However, a major drawback of this loss is its unstability if computed naively.

*  **TOP1 & BPR**

Additionnally, the authors propose two other pairwise losses that are either inspired (TOP1) or specific to ranking problems (BPR) and recommandation systems. The first one of these two functions is called TOP1 and defined as follows : 
$$
L_{top1} = \frac{1}{N_s} \sum \limits_{j=1}^{N_s} \sigma(r_j - r_i) + \sigma(r_j^2) 
$$
<center><i> where $\sigma(.)$ denotes the sigmoid function, $j \in {1,...N_s}$ are the indexes of the negative samples, $i$ denotes the index of a relevant sample </center></i>

Secondly, they use a loss based on the BPR-Opt criterion introduced in the paper [*BPR: Bayesian Personalized Ranking from Implicit Feedback*](https://arxiv.org/abs/1205.2618). This loss present the advantage of being directly optimized for ranking and has been developed for the task of item recommandation. This loss is defined as follows with the previous notation still valid : 
$$
L_{BPR} = - \frac{1}{N_s} \sum \limits_{j=1}^{N_s} log[\sigma(r_i-r_j)]
$$

*  **TOP1 & BPR losses : the problem of vanishing gradient**

Both of the proposed loss functions have a major drawback. Indeed, they are listwise loss functions which, for each target, compute the average of pairwise loss functions (one for each negative example considered). Thus by taking this average, they attribute the same weight to each of the training negative example. However, all negative example are not relevant. As reminded before, it is important with these losses that the score of the target item does not exceed the score of the negative example by a too large margin, otherwise there is nothing to be learnt. Indeed, if it is the case then, the ranking between both items is already known. Therefore we want negative examples with a high score and if $r_j \ll r_i$ the negative example is not relevant. 

Let's make sure of what we have said. We will look at the gradients of both TOP1 and BPR ranking losses with respect to a target score as we learn parameters of the model from them. 

Let's start with the TOP 1 : 
$$
\frac{\partial L_{top1}}{\partial r_i} = - \frac{1}{N_S} \sum \limits_{j=1}^{N_S} \sigma'(r_j - r_i)
$$
Then, for a negative sample, 
$$
r_j \ll r_i \Rightarrow r_j - r_i \ll 0 \Rightarrow \sigma'(r_j - r_i) = 0
$$
Then for the BPR : 

$$
\frac{\partial L_{BPR}}{\partial r_i} = - \frac{1}{N_S} \sum \limits_{j=1}^{N_S} (1-\sigma(r_i - r_j))
$$
Then, for a negative sample, 
$$
r_j \ll r_i \Rightarrow 0 \ll r_i - r_j \Rightarrow \sigma(r_i - r_j) = 1 \Rightarrow (1 - \sigma(r_i - r_j)) = 0 
$$

We can therefore see that the model doesn't learn from the irrelevant examples. However, by taking the average in the loss function, an irrelevant negative sample weights as much as a relevant one since the gradient is discounted by the total number of samples. Moreover, for a precise item, as the number of negative examples considered increases, the number of irrelevant ones increases faster than the relevant ones. Therefore we have gradient which vanishes as the number of samples increases. This explains why we want *relevant* negative examples (i.e. high scoring negative examples) and if $r_j \ll r_i$ then the negative sample $j$ is not relevant. 

### The ranking-max loss function family

To avoid vanishing gradients, the two authors propose to theoretically drop the computation of pairwise losses between the target score and each negative examples before taking the average. Instead, they propose to directly compare the score of the considered item (target) with the highest-scoring negative sample. This is formulated by the authors as follows : 
$$
L_{pairwise-max}(r_i, \{r_j\}_{j=1}^{N_S}) = L_{pairwise}(r_i, \max_{j=1,\dots, N_s}(r_j))
$$

They justify rewriting the BRP-max and the TOP1-max based on this idea and using a re-weighting scheme as a differentiable approximation of the maximum. Indeed, the maximum function used by the authors as above is not differentiable. Intuitively, as we are looking in our problem for the highest scoring item, a smooth approximation to the maximum function is indeed a preferable choice. Thus, they propose to use the softmax function as an approximation, defined as follows : 
$$
s_i = \frac{e^{r_i}}{\sum \limits_{j=1}^N {e^{r_j}}}
$$

<center><i>where $r_i$ is the  score of the considered item and $N$ is the set of the batch items. </center></i>

Hidasi and Karatzoglou propose to use the softmax score computed for each considered negative sample as weight in the average of the pairwise evaluations. That is to say, to consider the negative sample with the highest score as exposed in their pairwise-max family, each negative sample weight is proportional to its likelihood of being the highest scoring. This theoretical improvement gives two new losses : 

*  **The TOP1-max :** 
$$ 
L_{TOP1-max} = \sum \limits_{j=1}^{N_s} s_j [\sigma(r_j - r_i) + \sigma(r_j^2)]  
$$

*  **The BPR-max :**
$$
L_{BPR-max} = - log \sum \limits_{j=1}^{N_s} s_j \sigma(r_i-r_j)
$$

Then, we remind that the interest of defining these functions is to prevent the vanishing gradient that we had. If we look at the gradient of the newly defined TOP1-max and BPR-max with respect to the targeted item, we have :

$$
\frac{\partial L_{TOP1-max}}{\partial r_i} = - \sum \limits_{j=1}^{N_s} s_j \sigma(r_j - r_i) [1 - \sigma(r_j - r_i)]
$$

$$
\frac{\partial L_{BPR-max}}{\partial r_i} = - \frac{\sum \limits_{j=1}^{N_s} s_j \sigma(r_i - r_j) [1 - \sigma(r_i - r_j)]}{\sum \limits_{j=1}^{N_s} s_j \sigma(r_i - r_j)}
$$

In both cases, the more irrelevant the negative sample is, the closer to zero its softmax score will be. Thus, by adding negative samples, even if more irrelevant samples are added, the weighting with the softmax score enables the model not to learn from those examples as their weights within the gradient are proportional to their relevance. 

**A last improvement : BPR-max with regularization**

Based on their experimental observations, Hidasi and Karatzogou noticed that the regularization part in the TOP1 and TOP1-max loss, i.e. the $ \forall j \in [1, ..., N_S], \sigma(r_j ^2)$ term, enabled to perform better than BPR and BPR-max. They propose a last improvement for the BPR-max with a regularization term which is the $l_2$ regularization for the negative samples' score with a softmax weight : 
$$
L_{BPR-max} = - log \sum \limits_{j=1}^{N_s} s_j [\sigma(r_i-r_j) + \lambda r_j^2]
$$

## Discussion on the  introduced improving procedures

### Sampling and Loss

Hidasi and Karatzoglou addressed two points: **sampling** and **loss**. In this section, we will argue that those two points are two sides of the same coin. To understand the loss, the sampling is needed, and vis-versa.
A good reference for sampling losses in the RNN context is [Blackout](https://arxiv.org/pdf/1511.06909.pdf). We also refer to [this document](https://www.tensorflow.org/extras/candidate_sampling.pdf), which explains clearly the link between the loss function, the sampling and what is actually learnt by the network. 

Hidasi and Karatzoglou use **sampled** losses. The term *cross-entropy* in their paper refers not to the usual cross-entropy, but to a version of it applied only to a subset of sampled items. The same is true for the rest of the losses they introduce. A **sampled** loss is different to its **full** counterpart. 

**Cross-entropy example**

Lets consider the cross-entropy 
$$L_{ce-softmax} = -log[p(y|f_\theta(x))]$$
where $y$ is the correct class (correct item to predict) and $f_\theta(x)$ is the output of the network for the input $x$. 

$$
\begin{align}
log[p(y|f_\theta(x))] &= log \;s_i \\
&= log \frac{e^{r_i}}{\sum \limits_{j=1}^N {e^{r_j}}} \\
&= r_i(\theta) - Z(\theta)
\end{align}
$$

<center> <i> where $Z = log \sum_j e^{r_j}$ is the normalization constant to obtain probabilities from raw values and $\theta$ the parameters of the network. </center> </i>

We can express the gradient of the cross-entropy as follows:

$$
\begin{align}
\frac{\partial L_{ce-softmax}(\theta)}{\partial \theta} = \frac{\partial r_i(\theta)}{\partial \theta} - \mathbb{E_{p_\theta(y|x')}}(\frac{\partial r(\theta)}{\partial \theta})
\end{align}
$$

<center> <i>With some slight abuse of notation. (See <a href="https://arxiv.org/pdf/1511.06909.pdf"> Blackout </a> for the full derivation).</center> </i>

Let $\mathcal{Q}$ be the distribution used for sampling, $\mathcal{Q}(x)$ is the probability of $x$ being sampled. The authors consider $\mathcal{Q} = supp^{\alpha}$. By choosing $\mathcal{Q}$, we effectively change the derivative of the loss to:
 $$
\begin{align}
\frac{\partial L_{ce-softmax-sampled}(\theta)}{\partial \theta} = \frac{\partial r_i(\theta)}{\partial \theta} - \mathbb{E_{\mathcal{Q}}}(\frac{\partial r(\theta)}{\partial \theta})
\end{align}
$$

The point is introducing a sampling adds a bias to the loss. It is possible to get rid of this bias by adding a correction term $-log\;\mathcal{Q}(x)$. 


**Link to the article**

The article has a main chapter about sampling, and another one about the design of losses. It even cites the  <a href="https://arxiv.org/pdf/1511.06909.pdf"> Blackout </a> paper, which explores the impact of non-uniform sampling on losses and the need to consider this impact when designing the losses. The authors had all the elements to propose sampling-aware losses and avoid the bias term introduced by the sampling. We way wonder why did they not went through with it.

### Ranking-Max losses

Hidasi and Karatzoglou define the family of max-losses by 

$$
L_{pairwise-max}(r_i, \{r_j\}_{j=1}^{N_S}) = L_{pairwise}(r_i, \max_{j}(r_j))
$$

They justify rewriting the BRP-max and the TOP1-max using a re-weighting scheme as a differentiable approximation of the maximum. 

**Problems with this approach.**

Hidasi and Karatzoglou state that the reason for using softmax-weights instead of just taking the max is because max is not differentiable. 

This is not a good reason. The max fonction can be used in most automatic differentiation frameworks. The gradient is only retropropagated for the maximum value and set to zero for the others. For instance, the max-pooling layer is widely used in convolutional networks without any trouble. (See [pytorch Max-pooling](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html)). Therefore it seems legitimate to ask if the authors truly could not use the max function as it seems possible in today's deep-learning frameworks. Maybe there was a confusion between the $max$ and $argmax$ function: the latter is indeed not supported by auto-differentiation frameworks. 

What is more, the choice of re-weighting each term of the loss according to the softmax of $r_j$, ie
$$
L_{max}(r_i, {r_j}) = \sum_j softmax(r_j) \mathcal{l}(r_i, r_j)
$$
is not obvious. By reweighting using the softmax, the maximum will indeed have a higher score and account for most of the lost. However, it does not follows the natural stream of thought. If your problem is that $max$ is not differentiable, the first solution that comes to mind is to replace it by a differentiable approximation. I would for instance use the The $logSumExp$ function. ([logSumExp wiki](https://en.wikipedia.org/wiki/LogSumExp)):

$$
L_{max}(r_i, {r_j}) =  L_{pairwise}(r_i, logSumExp_j(r_j))
$$

Considering this elements, we remained a bit puzzled by the introduction of max-loss functions family. Thoses losses did bring improvements over the previous ones, so the empirical results justify them a posteriori. However, the way they are introduced is not fully convincing, and a robust explanation would need to justify why not simply use the max function directly, as is it done nowadays, and also discuss the smooth approximation of the maximum.

**Other max-loss functions.**

The max-loss family proposed is in fact pretty general. We can observe that the cross-entropy is also a max-loss function.

$$
\begin{align}
L_{ce-softmax} &= -log(p(y|f_\theta(x))) \\
&= r_i(\theta) - log \sum_j e^{r_j} \\
&= r_i(\theta) - logSumExp_j(r_j)
\end{align}
$$

As seen above, the $LogSumExp$ is a smooth approximation of the maximum. We can interpret the cross entropy loss as the difference between $r_i$ and  the smooth maximum of the $r_j$. As such, the cross-entropy is a max-loss function too. 
With a slight twist of mind, a hinge loss can also be seen as a max-loss function, where $max_j(r_j) \approx r_j > margin$.

This fact may explain why the cross-entropy loss still performed okay compared to the other max-loss function they introduced. Why migh also ask why consider new losses when the cross-entropy already satisfied the max-loss principle. However, the empirical results tell us that theses newly designed losses were relevant, as they outperformed the cross-entropy one. Maybe because they were inspired be ranking problems. Maybe because they incorporated a regularisation term while the cross-entropy did not. We may wonder how would a regularized cross-entropy perform.

### Appendix : Further discussion remarks

**Blackout**

[Blackout](https://arxiv.org/pdf/1511.06909.pdf) is a sampling loss similar to what is proposed by Hidasi and Karatzoglou.

Shihao Ji, et al. called their sampling scheme blackout to play on the similarity with dropout. This highlights the regularizing nature of the output-sampling procedure. By using a sampling loss, Hidasi and Karatzoglou are also implicitly regularizing their network. Sadly their paper does not have any train-test performances comparisons. As said before, Hidasi and Karatzoglou cite this paper but strangely do not implement it when considering a sampled approximation of the softmax.

**Word2Vec**

We made multiples remarks linking GRU4Rec to Word2Vec. This is not a coincidence. It is highly probable that the authors were inspired by word2vec. For instance, you can see a conference where Karatzoglou discuss session-aware recommander systems and word2vec [here](https://www.youtube.com/watch?v=hDjEd43R7Ik).


**Score weight**

In a recommender system, the metrics we are interested in cannot be used directly as a loss function. Therefore, the loss function used is a proxy for the optimization of the metrics. A good loss whould provide  a tight bound over the desired metrics. This opens the discussion for the choice and design of losses in the particular case of recommander systems.

We refer to [Wei Chen & al.](https://papers.nips.cc/paper/2009/file/2f55707d4193dc27118a0f19a1985716-Paper.pdf) for a study on the link between the loss function and the metric in ranking systems. They provide a theoretical view of why optimizing a classical loss, such a the cross-entropy, lead to maximizing the metrics such as the NDCG. This gives a theoretical motivation for the design of a specific loss in the case of recommander systems.
In addition, they propose a re-weighting scheme to make the loss a tighter bound of the NDGC. Interestingly, the design of a specific loss and  the re-weighting is also the approach taken by Hidasi and Karatzoglou with the ranking-max-losses. Thus, Wei Chen & al. might provide a theoretical view of why BPR-max performed better than the standard cross-entropy.

* Wei Chen & al:

$$L_p(f,x,L) = \sum_{s=1}^{n-1}G(l(y(s)))\,D(1+\sum_{k=l(y(s))+1}^{K-1}n_k) \sum_{i=s+1}^{n}a(y(i),y(s))\phi(f(x_{y(s)}) - f(x_{y(i)}))$$

where:
* $\phi$ is the logistic function in gru4rec.
* $G$ is the gain function in the NDGC. For gru4rec, which uses the MRR, this is the identity.
* $D$ is the discount function in the NDGC. For gru4rec and the MRR, this is   $\frac{1}{rank}$. (in the NDGC it is commonly logarithmique: $\frac{1}{log(1+rank)}$. It penalises having a relevant document at a lower score than what it should be.
* $f$ is the model (gru4rec).
* $L$ are the labels. $l(i)$ is the label i. in gru4rec they are binary label (next or not).
* $y$ is a permutation; $x_{y(i)}$ is the $x$ at the $i^{\text{th}}$ in the ranking defined by y. In addition, if $l(i) \ge l(j)$ then $x_i$ is ranked above $x_j$ in y. (similar to the positive / negative samples in gru4rec); 
* $a(y(i), y(s))$ is the indicatrice function of $y(i) \ne y(j)$. In gru4rec, this is the indentity.
* K is the maximum rating. 
* $n_k$ is the number of items with label $k$.

*the NDGC is a generic metric used in recommander systems and information retreival. In particular, the MRR and REC metrics used by Hidasi et al. are specific cases of the NDGC*.


Overall, the sum over s represent the sum over all the outputs items. The coefficient $G(\,\cdot\,)$ is a positive weight proportional to the relevance of the currently considered output $s$. The coefficient $D(\,\cdot\,)$ is a positive weight also proportional with the relevance of s. The sum from $s+1$ to $n$ is the loss for all items that have a smaller label than s.

We can compare to:

$$L_{bpr-max} =  - log(\sum_{j=1}^{N_s} s_j\sigma(r_i-r_j))$$

Both losses have a re-weighting scheme.[Wei Chen & al.] propose to 
push up the weight when computing the loss for a high scoring item, which is a bit different from the re-weighting proposed by Hidasi & al.

# Implementation and Results

In this third part of our study, we propose to present, evaluate and discuss the implementation and experiements that Hidasi and Karatzoglou display in their paper. We will also redo part of the simulation study and present our work. Finally, as additional personal input and since the authors have implemented their solution on an online video portal, we propose a broad comparison of session-based recommandation algorithms depending on the task and datasets showing the diversity and complexity of this field as well as its large industry applications possibility and potential. 

## Datasets presentation and evaluation metrics

The authors propose to evaluate their improvements on four different datasets. 


*   RecSys Challenge 2015 dataset (RSC15) : clicks and buy events from an online webshop. Only the click data was considered.  
*   VIDEO and VIDXL : watch events from an online video service
*   CLASS : item page view events from an online classified site 

The scenario used by the author is, as it has been presented earlier, the next item prediction. That is to say for a test session, the model predicts a probability distribution over the catalog for each item to be the next one. Then only the $k$ first item are kept and ranked. This ranking represents the order in which the $k$ highest scoring items are predicted to be the next item. The model iterates over the whole session. 

To evaluate the performance on these datasets, two evaluation metrics are considered : 

*   Recall@20 : It is the proportion of cases on all test cases, where the item predicted as next event is amongst the top-20 proposed items. 
*   MRR@20 (Mean Reciprocal Rank) : average of reciprocal ranks of predicted items. The reciprocal rank is equal to zero if the predicted item is ranked above 20. 

The advantage of MRR over Recall is that MRR is taking into consideration the rank which can be important in some cases. Recall, on the contrary, doesn't take into account this rank but is usually correlated with KPIs such as Click-Through-Rate enabling to predict the activity following events more precisely. 



## Main Results

*  **Additional samples**

Firstly, the authors test their proposed additional negative samples as it has been explained earlier. They present the results on the CLASS dataset. The baseline model used is the GRU4REC model presented in their earlier paper, the goal being to improve this very model. They test alongside the additional sampling, four losses to see if some respond better than others. Two earlier proposed losses, TOP1 and softmax cross entropy and the two new proposed losses, TOP1-max and BPR-max are therefore tested in 8 configurations : no additional negative samples, 32, 128, 512, 2048, 8192, 32768 and finally all items. The results are reminded in the below figure, taken from the paper itself. 

<center><img src="https://drive.google.com/uc?id=1jw_q3z-l7J1IXDf52s-v9_ZMGZnp9RNo" width="400px" align="center"></center>  

On this figure, we obviously see that although a slight increase in recall performance is encountered with the TOP1 loss with a few extra negative samples, it quickly collapses. This could be explained by the vanishing gradient problem explained earlier : as we increase the number of negative samples, the number of irrelevant samples increases the most causing the model to stop learning because of the loss design. The three other losses improve better with additional negative samples with slight differences : cross entropy gains start to stabilize from 2048 additional samples while performance of TOP1-max records a stronger increase and score until that point but starts to decrease after. BPR-max performance increases all the way (apart from using all samples). Thus, we can see that using additional samples along with the newly proposed improved losses, in particular BPR-max, leads to an increased performance. 


**Remark** : An additional experiment is the potential additional training time and computational cost which is apparently rather negligible thanks to GPUs according to the author's results. 

*  **Improved loss functions**

In a rather complete table, authors present their results on the four datasets with each time seven model tested for both metrics introduced : a kNN model (baseline model for next item prediction), GRU4REC with TOP1 loss, GRU4REC with cross entropy and then GRU4REC with additional samples already presented with TOP1, Cross Entropy, TOP1-max and BPR-max. The main results of this tables are the following : 

*  As said before, additional sampling is not very beneficial for TOP1 with performance ranging from -8.36% to +5.43% depending on the datasets and metrics. Comparatively and as said earlier, additional samples improve significantly the model for cross-entropy loss. 

*  Mostly, the combination of BPR-max and additional samples brings a steady increase. Between 25% and 55% compared to the baseline kNN and between 18% to 37.5% compared to the original GRU4REC (no additional sample and TOP1). However the difference between cross-entropy and BPR-max with additional samples is not such remarkable, questioning the use of BPR-max (much more complex and computationally costly in time). 

Complete results as shown in the paper are presented in the following table : 

<center><img src="https://drive.google.com/uc?id=1QaAm6nHIY3GIXT-f1Gh3x2nky9WqptK9" width="800px" align="center"></center>  

**Remark** : Lastly, GRU4REC was used in an A/B online testing where the A version of an online video portal was proposing recommandations with a baseline complex logic approach, version  B and C were proposing recommandations thanks to two versions of GRU4REC. After 2.5 months of experiment, versions of the online video portal which used GRU4REC performed better on selected industrial KPIs (Watch time, clicks...) than the baseline model. 

# Implementing 

This notebook is intended to run on colab using GPUs.

Ensures first that you have downloaded the [RSC15 data](https://2015.recsyschallenge.com/challenge.html), and preprocessed it according to [this script](https://github.com/hidasib/GRU4Rec/blob/master/examples/rsc15/preprocess.py). You should store the results in your drive.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import tqdm as tqdm
%load_ext autoreload
%autoreload 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#main parameters.

batch_size = 32
test_batch_size = 1024 # we can use a higher batch size for testing since we are not interested in the stochatic properties of mini-batching.
hidden_size = 100
n_neg = 1024

## Data loading utillities.

We need to load large amount of sessions data in a compressed format.

In [ ]:
""" data loading utilities.
"""
import numpy as np
import itertools
import functools
import operator
import pandas as pd
import dataclasses

@dataclasses.dataclass
class SessionData():
    """
    A reprentation of a a bunch of sequences. close to a CSR matrix representation.
    sessions i is at data[off[i]:[i+1]].
    voc[i] is the item id at index i
    freq[i] is the associated frequency.
    """
    data: np.ndarray
    off: np.ndarray
    voc: np.ndarray
    freq: np.ndarray
        

def load_df(path, cols=("SessionId", "ItemId")):
    """loads the training dataset
    from the preprocessed file.
    """
    return pd.read_csv(path, sep="\t", usecols=cols, dtype=np.int64, memory_map=True).to_numpy().T

def offset(sessions):
    """get sessions offsets: session[i] = data[offsets[i]: offsets[i+1]]"""
    _, offsets = np.unique(sessions, return_index=True)
    return np.append(np.sort(offsets),sessions.shape[0]) #append lenght at the end so that offests[-1] gives the last element of sessions.

def vocabulary(items):
  """get the vocabulary (index to item id mapping), the translated data, and the frequencies.
  returns:
  -------
  voc: voc[i] is the event id associated with index i.
  items: the encoded data. items[i] is the index associated with the ith event when all sessions are concatenated.
  freq: freq[i] is the frequence (count) of event of index i.
  """
  voc, items, freq = np.unique(items, return_counts=True, return_inverse=True)
  return items.astype(np.uint16) , voc, freq #uint16: max vocabulary size is 65535.



def pprint(array,  width = 16):
  """ utility to print 2D arrays for testing purposes.
  """
  array  = np.atleast_2d(array)
  fmt = f"<{width}d"
  mke = lambda e: f"{e:{fmt}}"
  mkline = lambda l: functools.reduce(operator.add, (list(map(mke, l)))) + "\n"
  box = functools.reduce(operator.add, (list(map(mkline, array))))
  print(box)


def load(path)->SessionData:
    """
    loads all the necessary data from the dataframe.
    returns:
    a SessionData
    """
    data = load_df(path)
    of = offset(data[0])
    it, voc, freq = vocabulary(data[1])
    return SessionData(data=it, off=of, voc=voc, freq=freq)

## Model and Datasets

The pytorch model uses the simplified architecture implemented in the official GRU4REC.

Two datasets, to provide sessions-aware batches and random samples are also built. The implementation is also inspired from the official one. In particular, it aims at providing a fast iteration scheme.

In [ ]:
""" pytorch gru4rec model with tied weights.
"""
import torch
import torch.nn as nn
import numpy as np

class GRU4Rec(nn.Module):
  """GRU4rec with tied weights  using the minimal gated unit similar to what is done in the 
  GRU4Rec implementation."""
  
  def __init__(self, input_size, hidden_size):
    """
    parameters:
    ------
    input_size: number of words. (events)
    hidden_size: hidden dimention.
    """
    super().__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size

    # minified GRU cell with only 4 weights matrices
    self.Wx = nn.Parameter(torch.empty((input_size, hidden_size)))
    self.Wr = nn.Parameter(torch.empty((hidden_size, hidden_size)))
    self.Wh = nn.Parameter(torch.empty((hidden_size, hidden_size)))
    self.Wy = self.Wx # tied weights: the output projection matrix is the transpose of the input.
    
    self.bh = nn.Parameter(torch.empty((hidden_size,)))
    self.by = nn.Parameter(torch.empty((input_size,)))

    nn.init.xavier_normal_(self.Wx)
    nn.init.xavier_normal_(self.Wr)
    nn.init.orthogonal_(self.Wh) #orthogonal initialisation for the recurrent kernel.
    nn.init.uniform_(self.bh)
    nn.init.uniform_(self.by)
    
  def forward(self, inputs, carry, targets=None):
    """
    returns next carry and scores for the given inputs and targets.
    inputs:
    -----
    inputs: (bs,) index of the current items.
    carry: (bs,n_h) reccurent state of the GRU
    targets: (n_targets,) indexes of the items against which to compute scores.
    returns:
    -----
    h: (bs, n_h) cell state
    y: (bs, n_targets) scores
    """
    # gru
    s = self.Wx[inputs]
    v = s + self.bh
    r = torch.sigmoid(torch.addmm(v, carry, self.Wr))
    h = torch.tanh(torch.addmm(v, r * carry, self.Wh))
    h = (1-r) * carry + r * h
    if targets is not None: #partial score computation for using negative samples.
        y = torch.addmm(self.by[targets], h, self.Wy[targets].T)
    else: # full score computation for testing.
        y = torch.addmm(self.by, h, self.Wy.T)
    return h, y
  
  
  
class SessionBatchDataset(torch.utils.data.IterableDataset):
  """a fast batching dataset.
  """

  def __init__(self, data:SessionData, bs=32, random=True):
    super().__init__()
    self.data = data.data.astype(np.long) #need long dtype in pytorch.
    self.offsets= data.off
    self.bs = bs
    self.random = random
    
  def __iter__(self):
    r = np.arange(len(self.offsets)-1) # r is a permutation array. we access the offest array indirectly by first indexing in r. The allows shuffling of sessions.
    if self.random:
        np.random.shuffle(r)
    n_done = 0 # how many sessions have been fully seen.
    n_new = self.bs # how many new sessions to fetch to form the current batch.
    reset = np.ones(self.bs, dtype=bool) # true if the current element if the last of the sessions and we need to reset the associated cell state.
    CUR = np.zeros(self.bs, dtype=np.intp) # pointers to the current events
    END = np.zeros_like(CUR) # pointers to the last events of the current sessions.
    ft = np.arange(2, dtype=np.intp).reshape((-1,1)) # [0,1] array, used as a convenience to fetch the targets (the next items.)
    
    while n_new + n_done + 1 < r.shape[0]: # as long as we can have full batches.
      CUR[reset] = self.offsets[r[n_done:n_done+n_new]] # where we finished a sentence, we fetch a new one, in the order given by r.
      END[reset] = self.offsets[r[n_done:n_done+n_new] + 1] - 1 #offsets[x+1] is the first element of the sessions after x. offsets[x+1] - 1 is the last event of the session.
      batch = CUR + ft # form a batch of [features, targets] by looking for next item.
      b = self.data[batch] # form a batch of [features, targets] by looking for next item.
      reset = batch[1] == END # if the target is the last event of the session, we are done for this session.
      n_new = reset.sum() # compute how many to get for the new one
      n_done += n_new
      CUR += 1 # go to next element in all current sessions.
      yield b, reset
      
      
class RandomDataset(torch.utils.data.IterableDataset):
  def __init__(self,  data:SessionData, n_neg=1024, alpha=1.0):
    self.n_neg = n_neg
    self.alpha = alpha
    self.p = np.power(data.freq, alpha)# alpha-power raised support
    self.squash = 8 # distortion / speed trade-off by dividing the support. 
    self.p = np.ceil(self.p / self.squash).astype(np.int) # converting to inter valued intervals to enable fast sampling.
    self.expanded = np.repeat(np.arange(len(self.p), dtype=np.long), self.p) # memory time trade-off by materializing a probability table. 
  def __iter__(self):
      while True:
        yield np.random.choice(self.expanded,size=self.n_neg) # fast sampling: it is uniform sampling in our table.

### Utilities, losses and metrics.
implemented is BPR, Cross-Entropy.

In [ ]:
d = torch.arange(batch_size) # utility to get the diagonal.
def BPR(target):
    """ assuming that the bs first target are the true ones.
    inputs:
    -----
    target: (bs, n_targets). the first (bs, bs) values should be the in batch scores. As such, the first diagonal is the positives scores.
    """

    return -torch.mean(torch.nn.functional.logsigmoid(target[d,d].reshape((-1,1)) - target)) # using logsigmoid for numerical stability.
  
def CrossEntropy(target):
  """ batch sampled cross entropy
  inputs:
    -----
  target: (bs, n_targets). the first (bs, bs) values should be the in batch scores. As such, the first diagonal is the positives scores.
  """
  inputs = target
  target = torch.zeros_like(inputs)
  target[torch.arange(inputs.shape[0]),torch.arange(inputs.shape[0])] = 1 # first diagonal are the positive scores.
  return torch.nn.functional.binary_cross_entropy_with_logits(inputs, target)

In [ ]:
def F_MRR(target, scores, cutoff=20):
    """MRR @ 20
    target is of shape (bs,)
    scores is of shape (bs, |v|)
    """
    pos = scores[np.arange(len(target)),target].reshape((-1,1))
    ranks = (scores > pos).sum(axis=1) + 1 # if no scores are lower (ie best guess -> rank 1)
    return ((ranks <= cutoff)/ ranks).sum()

def F_RC(target, scores, cutoff=20):
    """recall @ 20
    target is of shape (bs,)
    scores is of shape (bs, |v|)
    """
    pos = scores[np.arange(len(target)),target].reshape((-1,1))
    ranks = (scores > pos).sum(axis=1) + 1 # if no scores are higher (ie best guess -> rank 1)
    return (ranks <= cutoff).sum()

def F_MRR_RC(target, scores, cutoff=20):
  """ a combined version of both metrics above"""
  pos = scores[np.arange(len(target)),target].reshape((-1,1))
  ranks = (scores > pos).sum(axis=1) + 1 # if no scores are higher (ie best guess -> rank 1)
  after_cut = (ranks <= cutoff)
  return (after_cut / ranks).sum(), after_cut.sum()

## Charging the data and building the model

We load the data from the drive since we are in colab to benefit from GPU acceleration. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# load CSR data. There is a bit a data to move around, takes a few seconds.
# replace with the location of your data.
data = load("/content/drive/MyDrive/datasets/processed/rsc15_train_tr.txt")
test_data = load("/content/drive/MyDrive/datasets/processed/rsc15_test.txt")

In [ ]:
# initialize sampler from the data.
test_batch_loader = torch.utils.data.DataLoader(SessionBatchDataset(test_data, bs=test_batch_size  , random=False), batch_size=None, pin_memory=True) # pin memory for faster batch transfert to GPU.
batch_loader = torch.utils.data.DataLoader(SessionBatchDataset(data), batch_size=None, pin_memory=True) #batch size none since manually batching to be fast.
random_loader = torch.utils.data.DataLoader(RandomDataset(data), batch_size=None, pin_memory=True)

In [ ]:
# build the model.
model = GRU4Rec(len(data.voc), hidden_size).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

## Trainning and evaluating.

Both are in the same loop to see if the training is going well or not.

In [ ]:
def train_loop(model, optimizer, loss):
    n_test = 1000 # nb of batch before each evaluation round.
    MRR = 0.0
    REC = 0.0
    carry = torch.zeros((batch_size, hidden_size)).to(device) # cell state initialized to zero.
    #tq = tqdm.notebook.tqdm((zip(batch_loader, random_loader))) #tqdm does not work in colab
    tq = zip(batch_loader, random_loader)
    for i, ((batch, reset), random) in enumerate(tq):
        batch = batch.to(device)
        reset = reset.to(device)
        random = random.to(device)

        target = torch.cat([batch[1], random]) # minibatch samples + negative samples. 
        carry, y = model(batch[0], carry, target)
        l = loss(y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        carry = carry.detach()
        carry[reset] = 0
        #tq.set_postfix_str(f"{l.detach().item():.2f} MRR {MRR:.2f} REC {REC:.2f}", refresh=False)
        if (i % n_test) == (n_test - 1):
          n_batch = 0
          MRR = 0.0
          REC = 0.0
          with torch.no_grad():
              test_carry = torch.zeros((test_batch_size, hidden_size)).to(device)
              for j, (batch, reset) in enumerate(test_batch_loader):
                  batch = batch.to(device)
                  reset = reset.to(device)
                  test_carry, y = model(batch[0], test_carry)
                  test_carry[reset] = 0
                  n_batch += 1
                  bMMR, bREC = F_MRR_RC(batch[1], y)
                  MRR += bMMR.cpu().item()
                  REC += bREC.cpu().item()
                  #tq.set_postfix_str(f"{l.detach().item():.2f} {j} MRR {MRR/n_batch:.2f} REC {REC/n_batch:.2f}", refresh=False)
              MRR /= (n_batch * test_batch_size)
              REC /= (n_batch * test_batch_size)
              #tq.set_postfix_str(f"{l.detach().item():.2f} MRR {MRR:.2f} REC {REC:.2f}", refresh=False)
          print(f"[{i:>8}] L:{l:.5f} MRR:{MRR:.5f} REC:{REC:.5f}")    

## Remarks on the Implementation

You can find the official implementation of GRU4Rec in [Hidasi's Github](https://github.com/hidasib/GRU4Rec).

We re-implemented part of their work in Pytorch.
Trying to implement and reading through their code proved instructive. The official implementation is in *Theano*, which is sadly discontinued. 

**Training speed**

Their implementation is quite impressive. 99% of the time is spent on GPU. This is important, because the volume of data is non-negligible. They are able to do 10 epochs of training in about one hour and a half. To understand how impressive that is, this number translate to more than 1'000 32-items batches per second.

Once you consider that you only have a milisecond to train on a batch, you understand the importance of a really fast sampling scheme.

This is an important issue for this domain where large datasets pose scalability issues. See for instance the [Dietmar's lecture](https://web-ainf.aau.at/pub/jannach/slides/IIR2018.pdf), highlighting the scalibity requirements.

**How to batch ?**

https://github.com/hidasib/GRU4Rec/issues/22

Making mini-batch is a complex issue, because ones need to deal with the various lenghts of sessions. The batching technique used by Hidasi is a bit unusual compared to what people use with RNN. Usualy, multiple tokens are fed at once from parallel sequences; the gradient is backpropagated through time on all thoses tokens. Additional padding is added for sequences that do not end simultaneously. 

Their implementation only use one token at a time. Below is a more classical version of the bppt batches.

Making mini-batches is the hard part of the implementation. Session data is conserved similarly to a Compressed Row Format matrix. This is important for both keeping the memory requirements low and enable a fast batching. Access to a session is done indirectly throught an offest array. In addition, one have to keep track of the state of each sessions to know when to reset the GRU state.


In [ ]:
def epoch(items, offsets, r):
      """ an iterator over all the batches in an epoch.
          drops the last incomplete batch.
          suffles sessions at the beginning of the batch.
          c -> current
          o -> offset
          s -> session
      """
      o = offsets
      i = items
      n_done = 0
      n_new = GRU4RECConfig.bs
      cs = np.empty(GRU4RECConfig.bs, dtype=np.int64)
      so = np.zeros_like(cs)
      reset = np.ones_like(cs, dtype=bool)

      #nota: it will leave the last sequences: this stops as soon as it cannot form a bs batch.
      while n_done + n_new <= o.shape[0]:
        cs[reset] = r[n_done : n_done + n_new] # fetches new sessions
        STARTS = o[cs] # index of the start of current sessions
        ENDS = o[cs+1] # index of the end of the current sessions.
        bo = STARTS + so + np.expand_dims(np.arange(GRU4RECConfig.bptt + 1),axis=1) # add current position in the sentence to get the final position.
        mask = (bo >= ENDS) # we need to mask session "that are past the end." 
        b = np.where(mask, 0, i[bo]) # masked items are set to zero.
        reset = mask[-1] # which sessions are done.
        so = np.where(reset, 0, so+GRU4RECConfig.bptt) # reset the session-offest for new sessions.
        n_new = reset.sum()
        n_done += n_new
        yield b, reset, mask
        

**GRU or GRU**

The implementation of the GRU cell in GRU4REC is not the standard one. They implemented a simplified one, avoid 2 matrix multiplication out of the six.

In addition, in their default configuration, the input embedding matrix is directly the input-hidden vector of the gru cell, avoiding yet another matrix multiplication. 

This simplification is essential to the fast performance of their implementation.

**Maybe they tried to implement blackout**

https://github.com/hidasib/GRU4Rec/blob/master/gru4rec.py#L496


When further inspecting the source code, we can see a ```logq``` parameter:
This logq parameter correspond to the correction one want to apply to the loss when sampling with a non-uniform distribution. It is a bit disapointing that this point appears in the code but not in the paper, as it would have been an intersting discussion. 

# Improvements, further research

One troubling fact is that we are learning using hard labels, ie, is the next predicted item the right one or not. But it is highly probable that in the same situation, multiple items deserve a high rank. It would be interesting to experiment with smoothing labels, perhaps by using the training set prior.

# Ouverture

GRU4REC was compared to other sessions based models in [this study](https://arxiv.org/pdf/1803.09587.pdf).

## GRU4REC performance in Session-based recommandation systems models evaluation 

This study expands the evaluation to new datasets. It includes 4 e-commerce datasets (RSC15, )TMALL, RETAILR, ZALANDO), composed of users actions on different e-commerce plateforms, 4 music sessions datasets (NOWPLAYING, 8RACKS, 30MUSIC, AOTM), using listening history and playlists, and 1 media dataset (CLEF), using articles access and publication. As we can see in the paper, several metrics are computed for each dataset and model : Hit rate (HR@20), Mean Reciprocal Rank (MRR@20), Catalog Coverage (COV@20), Popularity (POP@20). 

The comparison is done with different nearest-neighborhood methods (including item-knn), as well as well as some simple baseline models such as a first-order markov model, that simply predicts the most likely item given the previous one. 

GRU4REC performs relatively well and is always one of the top contenders in the differents metrics. Interestingly, it is the best performer only on the RSC15 dataset. It is possible that Hidasi and al. slightly overfitted on the RSC15 dataset.

However, the article points out that GRU4REC does not perform substancially better than the kkn-based methods, and that baseline methods are not far off too. Their recommandation is to first try a simple method to obtain a better idea of the task you have. It is interesting to note that training GRU4REC is extremely more complex (about 400% slower if using a GPU, even worse on CPU) that training a basic method. This gives another reason to first try basic methods.

In the case of the RSC15 dataset, about half of all sessions are only of lenghts 2. For all thoses extremely short sessions, the GRU does not bring any benefits compared to the baseline methods. This preponderance of short sentences might explain the relatively great performances of baseline methods.





## Session-based recommendation evaluation

What is also interesting to see is that there is no clear models/business applications couple. Indeed, we see that S-SKNN, S-KNN, V-SKNN perform well on two e-commerce datasets (namely RETAILR and TMALL) but are below for the two others (RSC15 and ZALANDO). As said, GRU4REC performs the best on RSC15 but not on others. Concerning music datasets, SR model seems to perform globally the best but once again not for evey dataset and GRU4Rec has globally top  performances but not always and is the best on no music datasets. On music datasets, V-SKNN, S-KNN and S-SKNN (good performers on e-commerce datasets) are closer to the bottom. For these remarks we used Table 3 (page 19) and Table 5 (page 23) of the paper citedd at the beginning of this section. Thus, we can say that although session-based recommandation systems is fragmented between different business applications (here music and e-commerce evaluated) there seems to be no model performing unanimously better across all applications (although GRU4REC is performing quite well) and for one application, there seems to be no specifically better model either even if we have some trends (V-SKNN for e-commerce vs SR for music for example). 

There are numerous points to take away from this evaluating paper. Firstly concerning the evaluation methods for (session-based) recommandation systems, we note a wide range of evaluation datasets. In this paper, 9 were used. Whereas image processing models are usually compared to the baseline MNIST dataset or object recognition use ImageNet, recommandation systems seem to lack a reference dataset for model evaluation and comparison. This could be explained by the numerous and  different use cases of session-based recommandation system and the need for a company (such as Zalando or Netflix) to come up with an efficient model only for their application and task. However it still calls for a reference baseline database. 

Secondly, the question of uniform evaluation methods and metrics for (session-based) recommandation systems seems to be of importance. In our studied paper and the evaluating one, we can note two broad evaluation methodds. Firstly, the traditional approach of testing on fixed datasets. Concerning this approach, a note is the numerous metrics used that are linked or not to business KPIs : Mean Reciprocal Rank, Recall, Precision, Coverage, Popularity, Hit Rate ...  Although depending on the task, one could be preferable than another and allow for clear choice between models, the lack of homogeneity in  metrics makes it difficult to call for a state of the art model. The second method seen is the A/B online testing. Indeed, the major drawback of fixed dataset evaluation is the difficulty to accurately predict the reactions of real users to the recommandations. Thus, traditional metrics on fixed dataset and the deducted effectivene will always be imprecise. In A/B tests, recommendations are shown to typically thousands of users of a real product, and the recommender system randomly picks at least two different recommendation approaches to generate recommendations. The effectiveness is measured with implicit measures of effectiveness such as conversion rate or click-through rate. This application methods for evaluating seems to be closer to the end purpose of the recommandation systems and allow for a real-world choice. 

## The evolution of recommandation systems 

In recent years, recommandation systems have greatly evolved from traditional matrix factorization and completion algorithms like the ones seen in the course to complex deep learing methods through baseline models (such as kNN). In this paper, we have studied an efficient implementation of GRU to the problem and it seems therefore relevant to wonder what could be the next evolution. 

Firstly, as presented in the previous paragraph comes the question of uniformizing the evaluation methods: Prefer A/B tests with business indicators chosen according to the task or even the company ? To evaluate the models in a general way, propose pairs of datasets / metrics by application (e-commerce, music, video, media ...) since it seems difficult to do so for all applications (a prevalent metric for all datasets).  

Secondly, a research domain that seems relevant to us are the cross-domain recommandation systems. Traditionnal recommender systems suggest items belonging to a single domain (e.g. movies for Netflix, music for Spotify etc.). It is also the case for session-based recommandation systems. Nowadays, users provide feedback for items of different types (e.g., in Amazon books, DVDs, …) and express their opinions on different social media. Digitalization has also allowed for a multiplication of players in every domain (e-retailers for instance). Moreover it seems relevant for companies to cross-sell products and services (e.g. Amazon : products, prime, music) and provide recommendations to new users beyond the ongoing session. Of course, we are here leaving the context of session-based recommandation system where the only data available are the events since the beginning of an unknown user's session. Cross-domain recommandation session was one of the topic 7 years ago at RecSys and seeems to have slowed down since then. However, we can wonder to what extent efficient cross domain recommandation systems could be leveraged to use data in a way similar to personalized retargeting (e.g. Criteo). 


# Conclusion 

Recommandation system have been an ever-increasing interest domain since the digitalization of consumers' habits to enable online companies to sell their products or services or retain their users with higher probability thus maximizing profit and sustaining their use. Compressed sensing through the low-rank and nuclear norm minimization problems propose a solution to the matrix completion approach for collaborative filtering. However deep learning has recently emerged as new dominant models in recommandation systems as in many other data science fields. In the context of session-based recommandation systems, which have grown in importance with an increase of new users, users not logged in etc., the sequential handling offered by RNN seems to be a "natural" way to model a user's sessions thus bringing efficient recommandations. 

In the studied paper, Hidasi and Karatzoglou propose improvements on their [previously introduced model](https://arxiv.org/pdf/1511.06939.pdf) GRU4Rec. Their contributions are:

* Using additional negative samples
* Proposing a new loss family
* Evaluation on 4 datasets

Their main contribution is to use additional samples, which is responsible for most of the performance improvement on various datasets. It is a simple and effective idea that is likely motivated by improvements in hardware performances that rendered the clever tactic they previously used unecessary and allowed for more negative samples.

Most of what is discussed in their section 2 (sampling), 3.1 (Categorical Cross-Entropy)  and section 3.2 (Ranking-losses) are remainders and do not introduce new concepts. The proposed new loss family "Ranking-max loss" is interesting, yet hindered by unsufficient explanations. Finally, sadly only one of the experiements has a publicly available dataset.

Considering the above points, and the the remarks on the implementation, mostly on how the focus was on optimizing the training time, it feels like this paper is a result of continuous work on improving on their GRU4Rec architecture, and not a discussion on loss function and sampling procedure.

Finally, the improved model GRU4REC is a performing model for session-based recommandation datasets. It is a top performer on different applications (e-commerce and music) and on several datasets for each of them. However, the outperformance of the model is not always singificant given the computational cost of complexity compared to baseline models. Moreover, the evaluation methods and metrics of (session-based) recommandation systems seem to suffer from a lack of determined datasets / metrics evaluation to differentiate models and even methods are criticized as A/B Testing and business KPIs as metrics seem more relevant. Lastly, cross domain recommandation systems seem relevant to us nowadays with the numerous products, services and social media that intertwine for a user understanding and appropriate items recommandation (and relevant for companies wanting to cross-sell). 

GRU4REC is an interesting systems to bring deep-learning to recommenders systems. It also deals with the complex nature of sequential data in an elegant way using sampling and associated losses. 
